普通版，没有算均值

In [2]:
import re
import pandas as pd
from IPython.display import display
import os
Dataset_list = ['nyu','KITTI', 'redweb','ETH3D','Ibims', 'VKITTI', 'Matterport3D', 'UnrealCV']

from datetime import datetime

# 获取当前日期
current_date = datetime.now()

# 格式化日期为 YYYYMMDD 格式
formatted_date = current_date.strftime("%Y%m%d")

excelfile_name = "/data1/Chenbingyuan/Depth-Completion/application/results_xlsx/"+ formatted_date +".xlsx"
# 检查文件是否存在
if os.path.exists(excelfile_name):
    # 删除文件
    os.remove(excelfile_name)
    print(f"文件 {excelfile_name} 已删除。")
# 原始数据
for Dataset in Dataset_list:
    file_path = "/data1/Chenbingyuan/Depth-Completion/g2_dataset/" + Dataset + "/result/logs_ablation.txt"

    try:
        with open(file_path, 'r') as file:
            raw_data = file.read()
        print("文件读取成功，内容已转化为字符串。")
    except FileNotFoundError:
        print(f"文件未找到：{file_path}")

    # 从原始文本中提取数据
    pattern = r"rz_sb_mar_(\w+)_([\d.]+): AbsRel= (\d+.?\d*), RMSE= (\d+.?\d*), iRMSE= (\d+.?\d*), MAE= (\d+.?\d*)"
    matches = re.findall(pattern, raw_data)
    # print(matches)
    # 将提取的数据转换为 DataFrame
    df_data = {
        "模型名称": [],
        "点云密度": [],
        "AbsRel": [],
        "RMSE": [],
        "iRMSE": [],
        "MAE": []
    }

    for match in matches:
        model_name, density, abs_rel, rmse, irmse, mae = match
        # model_name = model_name.replace('rz_sb_mar_','')
        if 'NLSPN_DIODE_HRWSI_60' in model_name:
            print(Dataset, density, rmse)
        df_data["模型名称"].append(model_name)
        df_data["点云密度"].append(float(density))
        df_data["AbsRel"].append(float(abs_rel))
        df_data["RMSE"].append(float(rmse))
        df_data["iRMSE"].append(float(irmse))
        df_data["MAE"].append(float(mae))

    all_density_df = pd.DataFrame(df_data)
    # 按照点云密度分组
    grouped_df = all_density_df.groupby("点云密度")
    # 展示 DataFrame
    # display(all_density_df)

    # 遍历每个分组并使用 IPython 的 display 函数展示其数据
    # for density, group in grouped_df:
    #     print(f"点云密度: {density}")
    #     display(group)

    # 创建一个空的DataFrame用于拼接
    concatenated_df = pd.DataFrame()

    # 遍历每个分组，将其添加到concatenated_df中
    # 遍历每个分组，将其添加到concatenated_df中
    for density, group in grouped_df:
        # 可以选择添加一行来标记每个分组的开始
        concatenated_df = pd.concat([concatenated_df, group], ignore_index=True)

    if os.path.exists(excelfile_name):
        with pd.ExcelWriter(excelfile_name, mode='a',engine='openpyxl', if_sheet_exists='replace') as writer:
            concatenated_df.to_excel(writer, index=False, sheet_name=Dataset)
    else:
        with pd.ExcelWriter(excelfile_name, mode='w',engine='openpyxl') as writer:
            concatenated_df.to_excel(writer, index=False, sheet_name=Dataset)
print(excelfile_name)


文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
/data1/Chenbingyuan/Depth-Completion/application/results_xlsx/20240529.xlsx


平均版↓

In [1]:
import re
import pandas as pd
import os
from datetime import datetime
Dataset_list = ['nyu','KITTI', 'redweb','ETH3D','Ibims', 'VKITTI', 'Matterport3D', 'UnrealCV']

# 定义要提取的指标
selected_metrics = ["RMSE", "MAE"]  # 你可以在这里添加或删除指标，例如 ["AbsRel", "RMSE", "iRMSE", "iMAE", "MAE"] 

# 定义要计算均值的点云密度
densities_set_1 = [0.01, 0.1, 0.2, 0.5, 0.7]
densities_set_2 = [1.04, 1.016, 1.064, 1.08, 1.032, 1.0128]

# 获取当前日期
current_date = datetime.now()

# 格式化日期为 YYYYMMDD 格式
formatted_date = current_date.strftime("%Y%m%d")

excelfile_name = "/data1/Chenbingyuan/Depth-Completion/application/results_xlsx/" + formatted_date + ".xlsx"
if os.path.exists(excelfile_name):
    # 删除文件
    os.remove(excelfile_name)
    print(f"文件 {excelfile_name} 已删除。")
# 创建一个空的DataFrame用于拼接所有数据集的结果
final_df = pd.DataFrame()

# 原始数据
for Dataset in Dataset_list:
    file_path = "/data1/Chenbingyuan/Depth-Completion/g2_dataset/" + Dataset + "/result/logs_ablation.txt"
    print(file_path)

    try:
        with open(file_path, 'r') as file:
            raw_data = file.read()
        print("文件读取成功，内容已转化为字符串。")
    except FileNotFoundError:
        print(f"文件未找到：{file_path}")

    # 从原始文本中提取数据

    pattern = r"rz_sb_mar_(\w+)_([\d.]+): AbsRel= (\d+.?\d*), RMSE= (\d+.?\d*), iRMSE= (\d+.?\d*), iMAE= (\d+.?\d*), MAE= (\d+.?\d*)"  # worked for all
    # pattern = r"rz_sb_mar_(\w+)_([\d.]+): AbsRel= (\d+\.\d+), RMSE= (\d+\.\d+), iRMSE= (\d+\.\d+), iMAE= (\d+\.\d+), MAE= (\d+\.\d+)"
    matches = re.findall(pattern, raw_data)
    # print(matches)
    # 将提取的数据转换为 DataFrame
    df_data = {
        "模型名称": [],
        "点云密度": []
    }
    
    metric_mapping = {
        "AbsRel": 2,
        "RMSE": 3,
        "iRMSE": 4,
        "iMAE": 5,
        "MAE": 6
    }

    for metric in selected_metrics:
        df_data[f"{Dataset}_{metric}"] = []

    for match in matches:
        print(match)
        # model_name, density, abs_rel, rmse, irmse, mae = match
        model_name, density, abs_rel, rmse, irmse, imae, mae = match
        if 'NLSPN_DIODE_HRWSI_60' in model_name:
            print(Dataset, density, rmse)
        df_data["模型名称"].append(model_name)
        df_data["点云密度"].append(float(density))
        
        for metric in selected_metrics:
            df_data[f"{Dataset}_{metric}"].append(float(match[metric_mapping[metric]]))

    dataset_df = pd.DataFrame(df_data)

    if final_df.empty:
        final_df = dataset_df
    else:
        final_df = pd.merge(final_df, dataset_df, on=["模型名称", "点云密度"], how="outer")

# 计算每个方法的均值
mean_df = final_df.groupby("模型名称").mean().reset_index()

# 计算指定点云密度集的均值
mean_df_set_1 = final_df[final_df["点云密度"].isin(densities_set_1)].groupby("模型名称").mean().reset_index()
mean_df_set_2 = final_df[final_df["点云密度"].isin(densities_set_2)].groupby("模型名称").mean().reset_index()

# 将最终的DataFrame写入Excel文件
with pd.ExcelWriter(excelfile_name, mode='w', engine='openpyxl') as writer:
    final_df.to_excel(writer, index=False, sheet_name="All_Datasets")
    # mean_df.to_excel(writer, index=False, sheet_name="Mean_Values")
    mean_df_set_1.to_excel(writer, index=False, sheet_name="随机点均值")
    mean_df_set_2.to_excel(writer, index=False, sheet_name="线数均值")

print(excelfile_name)


/data1/Chenbingyuan/Depth-Completion/g2_dataset/nyu/result/logs_ablation.txt
文件读取成功，内容已转化为字符串。
('sfv2_DIODE_HRWSI_LSM', '0.01', '0.0705287586569786', '10.3046', '0.013350', '0.003541', '6.6398')
('sfv2_DIODE_HRWSI_LSM', '0.1', '0.06490034193396571', '9.7626', '0.015157', '0.003686', '6.2870')
('sfv2_DIODE_HRWSI_LSM', '0.2', '0.0645007056236267', '9.7281', '0.015379', '0.003722', '6.2457')
('sfv2_DIODE_HRWSI_LSM', '0.5', '0.06332107732892038', '9.6455', '0.015458', '0.003701', '6.1730')
('sfv2_DIODE_HRWSI_LSM', '0.7', '0.06296374927163123', '9.6189', '0.015345', '0.003697', '6.1507')
('sfv2_DIODE_HRWSI_LSM', '1.04', '0.27170657625198363', '29.9663', '0.023367', '0.008374', '22.7137')
('sfv2_DIODE_HRWSI_LSM', '1.016', '0.2666248005867005', '29.7181', '0.023169', '0.008821', '22.5744')
('sfv2_DIODE_HRWSI_LSM', '1.064', '0.2698732961177825', '30.2391', '0.027301', '0.009215', '22.7181')
('sfv2_DIODE_HRWSI_LSM', '1.08', '0.28022784485816965', '31.0482', '0.028633', '0.010146', '23.0948')
('

汇总版本，有问题↓

In [31]:
import re
import pandas as pd
import os
from datetime import datetime
Dataset_list = ['nyu','KITTI', 'redweb','ETH3D','Ibims', 'VKITTI', 'Matterport3D', 'UnrealCV']

# 定义要提取的指标
selected_metrics = ["RMSE", "MAE"]  # 你可以在这里添加或删除指标，例如 ["AbsRel", "RMSE", "iRMSE", "MAE"]

# 定义要计算均值的点云密度
densities_set_1 = [0.01, 0.1, 0.2, 0.5, 0.7]
densities_set_2 = [1.04, 1.016, 1.064, 1.08, 1.032, 1.0128]

# 获取当前日期
current_date = datetime.now()

# 格式化日期为 YYYYMMDD 格式
formatted_date = current_date.strftime("%Y%m%d")

excelfile_name = "/data1/Chenbingyuan/Depth-Completion/application/results_xlsx/" + formatted_date + ".xlsx"
if os.path.exists(excelfile_name):
    # 删除文件
    os.remove(excelfile_name)
    print(f"文件 {excelfile_name} 已删除。")
# 创建一个空的DataFrame用于拼接所有数据集的结果
final_df = pd.DataFrame()

# 原始数据
for Dataset in Dataset_list:
    file_path = "/data1/Chenbingyuan/Depth-Completion/g2_dataset/" + Dataset + "/result/logs_ablation.txt"

    try:
        with open(file_path, 'r') as file:
            raw_data = file.read()
        print("文件读取成功，内容已转化为字符串。")
    except FileNotFoundError:
        print(f"文件未找到：{file_path}")

    # 从原始文本中提取数据
    pattern = r"rz_sb_mar_(\w+)_([\d.]+): AbsRel= (\d+.?\d*), RMSE= (\d+.?\d*), iRMSE= (\d+.?\d*), MAE= (\d+.?\d*)"
    matches = re.findall(pattern, raw_data)

    # 将提取的数据转换为 DataFrame
    df_data = {
        "模型名称": [],
        "点云密度": []
    }

    metric_mapping = {
        "AbsRel": 2,
        "RMSE": 3,
        "iRMSE": 4,
        "MAE": 5
    }

    for metric in selected_metrics:
        df_data[f"{Dataset}_{metric}"] = []

    for match in matches:
        model_name, density, abs_rel, rmse, irmse, mae = match
        if 'NLSPN_DIODE_HRWSI_60' in model_name:
            print(Dataset, density, rmse)
        df_data["模型名称"].append(model_name)
        df_data["点云密度"].append(float(density))
        
        for metric in selected_metrics:
            df_data[f"{Dataset}_{metric}"].append(float(match[metric_mapping[metric]]))

    dataset_df = pd.DataFrame(df_data)

    if final_df.empty:
        final_df = dataset_df
    else:
        final_df = pd.merge(final_df, dataset_df, on=["模型名称", "点云密度"], how="outer")

# 计算每个方法的均值
mean_df = final_df.groupby("模型名称").mean().reset_index()

# 计算指定点云密度集的均值
mean_df_set_1 = final_df[final_df["点云密度"].isin(densities_set_1)].groupby("模型名称").mean().reset_index()
mean_df_set_2 = final_df[final_df["点云密度"].isin(densities_set_2)].groupby("模型名称").mean().reset_index()

# 合并两个均值DataFrame
mean_df_set_1['类型'] = '随机点均值'
mean_df_set_2['类型'] = '线数均值'
combined_mean_df = pd.concat([mean_df_set_1, mean_df_set_2])

# 调整列顺序，将 '类型' 列放在第一列
cols = combined_mean_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
combined_mean_df = combined_mean_df[cols]

# 将最终的DataFrame写入Excel文件
with pd.ExcelWriter(excelfile_name, mode='w', engine='openpyxl') as writer:
    final_df.to_excel(writer, index=False, sheet_name="All_Datasets")
    combined_mean_df.to_excel(writer, index=False, sheet_name="均值汇总")

print(excelfile_name)


文件 /data1/Chenbingyuan/Depth-Completion/application/results_xlsx/20240607.xlsx 已删除。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
文件读取成功，内容已转化为字符串。
/data1/Chenbingyuan/Depth-Completion/application/results_xlsx/20240607.xlsx
